![Slide 1](./images/Slide2.JPG)


![Slide 2](./images/Slide3.JPG)


![Slide 3](./images/Slide4.JPG)


![Slide 4](./images/Slide5.JPG)


![Slide 5](./images/Slide6.JPG)


![Slide 6](./images/Slide7.JPG)


In [1]:
#!/usr/bin/env python3

def ask_multiline_question(query: str) -> str:
    """
    Ask a question that can span multiple lines.
    The user enters lines until a blank line is entered.
    """
    print(query)
    lines = []
    while True:
        line = input()
        if not line.strip():  # Stop reading on an empty line
            break
        lines.append(line)
    return "\n".join(lines).strip()


def ask_question(query: str) -> str:
    """Ask a single-line question and return the answer."""
    return input(query)


def run():
    # 1. SET UP REPORT TOPIC
    # Get initial multi-line query from the user
    initial_query = ask_multiline_question(
        "What would you like to research? (Enter blank line to finish)"
    )

    # Get breadth parameter (with default of 4 if input is empty or invalid)
    breadth_input = ask_question("Enter research breadth (recommended 2-10, default 4): ")
    try:
        breadth = int(breadth_input)
    except ValueError:
        breadth = 4

    # Get depth parameter (with default of 2 if input is empty or invalid)
    depth_input = ask_question("Enter research depth (recommended 1-5, default 2): ")
    try:
        depth = int(depth_input)
    except ValueError:
        depth = 2

    print("Creating research plan...")
    print(initial_query)
    print(f"breadth={breadth}, depth={depth}")


In [2]:
run()

What would you like to research? (Enter blank line to finish)


 How can I make fried bee hoon
 
Enter research breadth (recommended 2-10, default 4):  2
Enter research depth (recommended 1-5, default 2):  2


Creating research plan...
How can I make fried bee hoon
breadth=2, depth=2


In [14]:
import json
import asyncio
import os
from datetime import datetime
from pydantic import BaseModel, StrictStr, ValidationError
from typing import List
from dotenv import load_dotenv

load_dotenv(dotenv_path="./.env.local")


class FollowUpQuestions(BaseModel):
    questions: List[StrictStr]


def system_prompt():
    now = datetime.utcnow().isoformat()
    return (
        f"You are an expert researcher. Today is {now}. Follow these instructions when responding:\n"
        f"- You may be asked to research subjects that are after your knowledge cutoff, assume the user is right when presented with news.\n"
        f"- The user is a highly experienced analyst, no need to simplify it, be as detailed as possible and make sure your response is correct.\n"
        f"- Be highly organized.\n"
        f"- Suggest solutions that I didn't think about.\n"
        f"- Be proactive and anticipate my needs.\n"
        f"- Treat me as an expert in all subject matter.\n"
        f"- Mistakes erode my trust, so be accurate and thorough.\n"
        f"- Provide detailed explanations, I'm comfortable with lots of detail.\n"
        f"- Value good arguments over authorities, the source is irrelevant.\n"
        f"- Consider new technologies and contrarian ideas, not just the conventional wisdom.\n"
        f"- You may use high levels of speculation or prediction, just flag it for me."
    )


async def generate_feedback(query: str, num_questions: int = 3) -> List[str]:
    # Create the prompt message using a template.
    user_template = (
        f"Given the following query from the user, ask some follow-up questions to clarify the research direction. "
        f"Return a maximum of {num_questions} questions. User query: {query}"
    )

    messages = [
        {"role": "system", "content": system_prompt()},
        {"role": "user", "content": user_template},
    ]

    # Define the function specification for function calling.
    functions = [
        {
            "name": "generate_followup_questions",
            "description": "Generates follow-up questions for clarifying the research direction.",
            "parameters": {
                "type": "object",
                "properties": {
                    "questions": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of follow-up questions",
                    }
                },
                "required": ["questions"],
            },
        }
    ]

    model_name = os.getenv("OPENAI_MODEL", "gpt-4")
    # Instantiate the new async client.
    from openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    response = await client.chat.completions.create(
        model=model_name,
        messages=messages,
        functions=functions,
        function_call={"name": "generate_followup_questions"},
        temperature=0.5,
    )

    # Extract the function call result.
    message = response.choices[0].message
    if hasattr(message, "function_call") and message.function_call:
        arguments_str = message.function_call.arguments
        try:
            arguments = json.loads(arguments_str)
        except json.JSONDecodeError as e:
            raise ValueError("Failed to decode function call arguments") from e

        try:
            followup = FollowUpQuestions(**arguments)
        except ValidationError as e:
            raise ValueError("Validation error in parsed arguments") from e

        return followup.questions[:num_questions]
    else:
        raise ValueError("No function call found in the response.")


async def ask_multi_line_question(prompt_text: str) -> str:
    print(prompt_text)
    print("(Submit an empty line to finish your answer.)")
    lines = []
    while True:
        line = input()
        if not line.strip():
            break
        lines.append(line)
    return "\n".join(lines).strip()


In [15]:
initial_query = "can pigs fly"
breadth = 3
depth = 2

# Generate follow-up questions using the OpenAI API.
follow_up_questions = await generate_feedback(initial_query, breadth)

# Collect user answers for each follow-up question.
answers = []
for question in follow_up_questions:
    answer = await ask_multi_line_question(f"\n{question}\nYour answer:")
    answers.append(answer)

# Combine the initial query and the follow-up Q&A.
combined_query = f"Initial query: {initial_query}\n"
combined_query += "\n".join(f"Q: {q}\nA: {a}" for q, a in zip(follow_up_questions, answers))

print("\nPrinting User Prompt with Q & A...")
print(combined_query)
print(f"breadth={breadth}, depth={depth}")


/venvs/jupyter-venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1400: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


ValueError: `FollowUpQuestions` is not strict. Only `strict` function tools can be auto-parsed

In [21]:
import json
import asyncio
import os
from datetime import datetime
from pydantic import BaseModel, StrictStr, ValidationError
from typing import List
from dotenv import load_dotenv

load_dotenv(dotenv_path="./.env.local")


class FollowUpQuestions(BaseModel):
    questions: List[StrictStr]


def system_prompt():
    now = datetime.utcnow().isoformat()
    return (
        f"You are an expert researcher. Today is {now}. Follow these instructions when responding:\n"
        f"- You may be asked to research subjects that are after your knowledge cutoff, assume the user is right when presented with news.\n"
        f"- The user is a highly experienced analyst, no need to simplify it, be as detailed as possible and make sure your response is correct.\n"
        f"- Be highly organized.\n"
        f"- Suggest solutions that I didn't think about.\n"
        f"- Be proactive and anticipate my needs.\n"
        f"- Treat me as an expert in all subject matter.\n"
        f"- Mistakes erode my trust, so be accurate and thorough.\n"
        f"- Provide detailed explanations, I'm comfortable with lots of detail.\n"
        f"- Value good arguments over authorities, the source is irrelevant.\n"
        f"- Consider new technologies and contrarian ideas, not just the conventional wisdom.\n"
        f"- You may use high levels of speculation or prediction, just flag it for me."
    )


async def generate_feedback(query: str, num_questions: int = 3) -> List[str]:
    # Create the prompt message using a template.
    user_template = (
        f"Given the following query from the user, ask some follow-up questions to clarify the research direction. "
        f"Return a maximum of {num_questions} questions. User query: {query}"
    )

    messages = [
        {"role": "system", "content": system_prompt()},
        {"role": "user", "content": user_template},
    ]

    # Define the function specification for function calling.
    functions = [
        {
            "name": "generate_followup_questions",
            "description": "Generates follow-up questions for clarifying the research direction.",
            "parameters": {
                "type": "object",
                "properties": {
                    "questions": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of follow-up questions",
                    }
                },
                "required": ["questions"],
            },
        }
    ]

    model_name = os.getenv("OPENAI_MODEL", "gpt-4")
    # Instantiate the new async client.
    from openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    response = await client.chat.completions.create(
        model=model_name,
        messages=messages,
        functions=functions,
        function_call={"name": "generate_followup_questions"},
        temperature=0.5,
    )

    # Extract the function call result.
    message = response.choices[0].message
    if hasattr(message, "function_call") and message.function_call:
        arguments_str = message.function_call.arguments
        try:
            arguments = json.loads(arguments_str)
        except json.JSONDecodeError as e:
            raise ValueError("Failed to decode function call arguments") from e

        try:
            followup = FollowUpQuestions(**arguments)
        except ValidationError as e:
            raise ValueError("Validation error in parsed arguments") from e

        return followup.questions[:num_questions]
    else:
        raise ValueError("No function call found in the response.")


async def ask_multi_line_question(prompt_text: str) -> str:
    print(prompt_text)
    print("(Submit an empty line to finish your answer.)")
    lines = []
    while True:
        line = input()
        if not line.strip():
            break
        lines.append(line)
    return "\n".join(lines).strip()


In [22]:
initial_query = "can pigs fly"
breadth = 3
depth = 2

# Generate follow-up questions using the OpenAI API.
follow_up_questions = await generate_feedback(initial_query, breadth)

# Collect user answers for each follow-up question.
answers = []
for question in follow_up_questions:
    answer = await ask_multi_line_question(f"\n{question}\nYour answer:")
    answers.append(answer)

# Combine the initial query and the follow-up Q&A.
combined_query = f"Initial query: {initial_query}\n"
combined_query += "\n".join(f"Q: {q}\nA: {a}" for q, a in zip(follow_up_questions, answers))

print("\nPrinting User Prompt with Q & A...")
print(combined_query)
print(f"breadth={breadth}, depth={depth}")



Are you interested in understanding the physiological limitations that prevent pigs from flying?
Your answer:
(Submit an empty line to finish your answer.)


 Yes
 



Are you referring to any specific species of pig or the general pig family (Suidae)?
Your answer:
(Submit an empty line to finish your answer.)


 family
 



Are you interested in exploring genetic or technological interventions that could potentially enable pigs to fly in the future?
Your answer:
(Submit an empty line to finish your answer.)


 no
 



Printing User Prompt with Q & A...
Initial query: can pigs fly
Q: Are you interested in understanding the physiological limitations that prevent pigs from flying?
A: Yes
Q: Are you referring to any specific species of pig or the general pig family (Suidae)?
A: family
Q: Are you interested in exploring genetic or technological interventions that could potentially enable pigs to fly in the future?
A: no
breadth=3, depth=2
